In [1]:
%cd ..

/home/haito/kaggle/rsna-str-2/workdir


In [2]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append(".")
from src.factory import *
from src.utils import *
from sklearn.metrics import log_loss

In [3]:
DATADIR = Path("../input/rsna-str-pulmonary-embolism-detection/")

train = pd.read_csv(DATADIR / "train.csv")

pre = pd.read_csv(DATADIR / "split.csv")
train = train.merge(pre, on="StudyInstanceUID")

portion = pd.read_csv(DATADIR / "study_pos_portion.csv")
train = train.merge(portion, on="StudyInstanceUID")

z_pos_df = pd.read_csv(DATADIR / "sop_to_prefix.csv").rename(columns={'img_prefix': 'z_pos'})
train = train.merge(z_pos_df, on="SOPInstanceUID")


### train = train.query("fold == 0 or fold == 1")  # now I have fold0,1 only

studies = train.StudyInstanceUID.unique()

In [4]:
def get_pred(_path):
    res = load_pickle(_path)
    raw_pred = pd.DataFrame({
        "SOPInstanceUID": res["ids"],
        **res["outputs"],
    })
    return raw_pred

def calib_p(arr, factor):  # set factor>1 to enhance positive prob
    return arr * factor / (arr * factor + (1-arr))

In [5]:
# check
# oof_f0 = get_pred("output_yuji/b3_non_weight//oof_fold0_ep0.pkl")
# plt.hist( oof_f0.pe_present_on_image, bins=300 )
# plt.show()

In [6]:
# ! SET claibration value. Run src/oof_optpy to calculate the value 
oof_f0, fold0_calib_f = get_pred("output/final_image_level/oof_fold0.pkl"), 3.8250639579850194
oof_f1, fold1_calib_f = get_pred("output/final_image_level/oof_fold1.pkl"), 8.555037588568537
oof_f2, fold2_calib_f = get_pred("output/final_image_level/oof_fold2.pkl"), 4.374239635034443
oof_f3, fold3_calib_f = get_pred("output/final_image_level/oof_fold3.pkl"), 7.480972390526775
oof_f4, fold4_calib_f = get_pred("output/final_image_level/oof_fold4.pkl"), 5.002262078458348

# BAD
if False:  # pick best one which yields weighted-logloss after calib
    oof_f3, fold3_calib_f = get_pred("output/035_pe_present___448___apex/fold3_ep0.pt.valid.pickle"), 6.541753595870311
    oof_f4, fold4_calib_f = get_pred("output/035_pe_present___448___apex/fold4_ep0.pt.valid.pickle"), 3.8250639579850194
    
if True: ### ==== do calib for each fold
    oof_f0["pe_present_on_image"] = calib_p(oof_f0["pe_present_on_image"], fold0_calib_f)
    oof_f1["pe_present_on_image"] = calib_p(oof_f1["pe_present_on_image"], fold1_calib_f)
    oof_f2["pe_present_on_image"] = calib_p(oof_f2["pe_present_on_image"], fold2_calib_f)
    oof_f3["pe_present_on_image"] = calib_p(oof_f3["pe_present_on_image"], fold3_calib_f)
    oof_f4["pe_present_on_image"] = calib_p(oof_f4["pe_present_on_image"], fold4_calib_f)

oof = pd.concat([oof_f0, oof_f1, oof_f2, oof_f3, oof_f4]).rename(columns={'pe_present_on_image': 'pred0'})

train = train.merge(oof[['pred0', 'SOPInstanceUID']], on="SOPInstanceUID")  # add pred

In [7]:
train_copyed = train.copy()
train.columns

Index(['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID',
       'pe_present_on_image', 'negative_exam_for_pe', 'qa_motion',
       'qa_contrast', 'flow_artifact', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1',
       'leftsided_pe', 'chronic_pe', 'true_filling_defect_not_pe',
       'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate',
       'exam_type', 'fold', 'pe_present_portion', 'z_pos', 'pred0'],
      dtype='object')

In [8]:
""" feature engineer """

' feature engineer '

In [9]:
train = train.sort_values(['StudyInstanceUID', 'z_pos'])
train_current_z_pos = train.groupby('StudyInstanceUID')['z_pos'].shift(0)

### for i in range(1, 20):
for i in range(1, 10):

    train[f'pred0_pre{i}'] = train.groupby('StudyInstanceUID')['pred0'].shift(i)
    train[f'pred0_post{i}'] = train.groupby('StudyInstanceUID')['pred0'].shift(-i)

# NORMALIZED Z POS
z_max = train.groupby('StudyInstanceUID').z_pos.max().rename('z_pos_max')
train = train.merge(z_max, on='StudyInstanceUID')
train['z_pos_norm'] = train['z_pos'] / train['z_pos_max']
train = train.drop('z_pos_max', axis=1)

In [10]:
train.tail()

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,...,pred0_post5,pred0_pre6,pred0_post6,pred0_pre7,pred0_post7,pred0_pre8,pred0_post8,pred0_pre9,pred0_post9,z_pos_norm
1790589,fffda3f22362,39ca5eaafffe,29e855db7f2b,0,1,0,0,0,0,0,...,NaN,1.633664e-06,NaN,1.558988e-06,NaN,3.526268e-07,NaN,9.371637e-07,NaN,0.976190
1790590,fffda3f22362,39ca5eaafffe,f7ca277a66c2,0,1,0,0,0,0,0,...,NaN,4.497674e-07,NaN,1.633664e-06,NaN,1.558988e-06,NaN,3.526268e-07,NaN,0.982143
1790591,fffda3f22362,39ca5eaafffe,59714fd8dd25,0,1,0,0,0,0,0,...,NaN,6.327988e-07,NaN,4.497674e-07,NaN,1.633664e-06,NaN,1.558988e-06,NaN,0.988095
1790592,fffda3f22362,39ca5eaafffe,b33567349fae,0,1,0,0,0,0,0,...,NaN,4.913005e-06,NaN,6.327988e-07,NaN,4.497674e-07,NaN,1.633664e-06,NaN,0.994048
1790593,fffda3f22362,39ca5eaafffe,53d378d07811,0,1,0,0,0,0,0,...,NaN,2.411817e-06,NaN,4.913005e-06,NaN,6.327988e-07,NaN,4.497674e-07,NaN,1.000000


In [11]:
ids = [c for c in list(train) if 'UID' in c]
targets = [
    'negative_exam_for_pe',
    'indeterminate',
    'chronic_pe',
    'acute_and_chronic_pe',
    'central_pe',
    'leftsided_pe',
    'rightsided_pe',
    'rv_lv_ratio_gte_1',
    'rv_lv_ratio_lt_1',
]
other_targets = [c for c in list(train) if 'pe_present_on_image' in c]
### remove_cols = ['fold', 'path', 'weight', 'qa_contrast', 'qa_motion'] + targets + ids + other_targets
### remove_cols = ['fold', 'path', 'weight', 'qa_contrast', 'qa_motion'] + ['exam_type','flow_artifact','pe_present_portion', 'true_filling_defect_not_pe'] + targets + ids + other_targets
remove_cols = ['fold', 'path', 'weight', 'qa_contrast', 'qa_motion'] + ['exam_type','flow_artifact','pe_present_portion', 'true_filling_defect_not_pe'] + targets + ids + other_targets + ['z_pos']

features = sorted(list(set(list(train)) - set(remove_cols)))
print(features)

['pred0', 'pred0_post1', 'pred0_post2', 'pred0_post3', 'pred0_post4', 'pred0_post5', 'pred0_post6', 'pred0_post7', 'pred0_post8', 'pred0_post9', 'pred0_pre1', 'pred0_pre2', 'pred0_pre3', 'pred0_pre4', 'pred0_pre5', 'pred0_pre6', 'pred0_pre7', 'pred0_pre8', 'pred0_pre9', 'z_pos_norm']


In [12]:
features_copyed = features.copy()

In [13]:
def fobj(pred, data):
    true = data.get_label()
    label = 2*true - 1
    weights = data.weights
    response = -label / (1 + np.exp(label * pred))
    abs_response = np.abs(response)
    grad = response
    hess = abs_response * (1 - abs_response)
    return grad*weights, hess*weights

In [14]:
import torch
bce_func = torch.nn.BCELoss(reduction='none')

def feval2(preds, data):
    scores = bce_func(torch.FloatTensor(preds), torch.FloatTensor(data.label))
    scores = scores * torch.FloatTensor(data.weights)
    return 'weighted logloss', torch.mean(scores), False

In [15]:
import torch
bce_func_logit = torch.nn.BCEWithLogitsLoss(reduction='none')

def feval(preds, data):
    scores = bce_func_logit(torch.FloatTensor(preds), torch.FloatTensor(data.label))
    scores = scores * torch.FloatTensor(data.weights)
    return 'weighted logloss', torch.mean(scores), False

In [16]:
import torch
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [17]:
# features = ['pred']  # for test
features = features_copyed

In [18]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
import warnings
warnings.simplefilter('ignore')
import pickle

oof_preds_list = []
models_list = []
target = 'pe_present_on_image'

for i in range(1):
    print(f'=================={i}================')
    if i % 4 == 0:
        params = {'boosting_type': 'gbdt',
            'objective': 'binary',
#             'metric': 'None',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.1,
            'feature_fraction': 0.9,
            'max_depth': 15,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'verbose': 100,
            'early_stopping_rounds': 100,
            'verbose': -1,
            } 
    elif i % 4 == 1:
        params = {
            'max_depth': 4,
            'max_leave': int(0.2 * 2 ** 4),
            'reg_lambda': 1,
            'reg_alpha': 1,
            'subsamples': 0.8,
            'colsample_bytree': 0.7,
            'objective': 'binary',
            'min_data_in_leaf': 0,
            'boosting': 'gbdt',
            'metric': 'None',
            'learning_rate': 0.1,
                      }
    elif i % 4 == 2:
        params = {
            'num_leaves': 19, 
            'min_data_in_leaf': 160,
            'min_child_weight': 0.03,
            'bagging_fraction' : 0.7,
            'feature_fraction' : 0.8,
            'learning_rate' : 0.1,
            'max_depth': -1,
            'reg_alpha': 0.02,
            'reg_lambda': 0.12,
            'objective': 'binary',
            'verbose': 100,
            'boost_from_average': False,
            'metric': 'None',
        }  
    else:
        params = {
            'objective': "binary",
            'metric': 'None',
            'boost_from_average': "false",
            'tree_learner': "serial",
            'max_depth': -1,
            'learning_rate': 0.1,
            'num_leaves': 197,
            'feature_fraction': 0.3,
            'bagging_freq': 1,
            'bagging_fraction': 0.7,
            'min_data_in_leaf': 100,
            'bagging_seed': 11,
            'max_bin': 255,
            'verbosity': -1}    
        
    oof_preds = np.zeros(train.shape[0])
    val_results = {}
    models = []
    params['random_state'] = i
    iter = 100000
#     for n_fold, (trn_idx, val_idx) in enumerate(kf.split(train, train[target])):

    for n_fold in range(5):
    ### for n_fold in range(2):
        print(f'    ==============fold{n_fold}================')
        tr = train.query(f'fold != {n_fold}')
        val = train.query(f'fold == {n_fold}')
        trn_data = lgb.Dataset(tr[features], label=tr[target])
        trn_data.weights = tr.pe_present_portion.values
        val_data = lgb.Dataset(val[features], label=val[target])
        val_data.weights = val.pe_present_portion.values
        
        clf = lgb.train(params, trn_data, num_boost_round=iter, valid_sets=[trn_data, val_data], valid_names=['train', 'val'],
#                         verbose_eval=200, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
                        feval=feval, fobj = fobj, verbose_eval=2000, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results, )
        file = f'lgbs/lgb_seed{i}_fold{n_fold}.pkl'
        pickle.dump(clf, open(file, 'wb'))
        oof_preds[train.fold==n_fold] = clf.predict(val[features])
    oof_preds_list.append(oof_preds)

print(f'-------------------------------------------------------------------------roc_auc: {roc_auc_score(train[target], np.mean(oof_preds_list, axis=0))}')
print(f'----------------------------------------------------------roc_auc using raw pred: {roc_auc_score(train[target], train["pred0"])}')
print(f'------------------------------------------------------------------------------AP: {average_precision_score(train[target], np.mean(oof_preds_list, axis=0))}')
print(f'---------------------------------------------------------------AP using raw pred: {average_precision_score(train[target], train["pred0"])}')

lgb_oof = np.mean(oof_preds_list, axis=0)
train['lgb_preds'] = sigmoid(lgb_oof)


==================0================
    ==============fold0================
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[53]	train's weighted logloss: 0.0110919	val's weighted logloss: 0.0110957
    ==============fold1================
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[76]	train's weighted logloss: 0.0107546	val's weighted logloss: 0.0116312
    ==============fold2================
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > n

In [19]:
bce_func = torch.nn.BCELoss(reduction='none')

lgb_losses = bce_func(torch.FloatTensor(sigmoid(lgb_oof)), torch.FloatTensor(train['pe_present_on_image']))

### torch.mean(lgb_losses*train['weight'].values)
torch.mean(lgb_losses*train['pe_present_portion'].values).item()

0.011420365194671893

In [20]:
# no stacking result
lgb_losses = bce_func(torch.FloatTensor(train['pred0']), torch.FloatTensor(train['pe_present_on_image']))
torch.mean(lgb_losses*train['pe_present_portion'].values).item()

0.01322563879510154

In [22]:
raise "BELOW IS NOT USED FOR FINAL SUB"

# <<< BELOW IS NOT USED FOR FINAL SUB >>> stacking for PE_REPESNT -> POS_EXAM 

In [23]:
DATADIR = Path("../input/rsna-str-pulmonary-embolism-detection/")

train = pd.read_csv(DATADIR / "train.csv")

pre = pd.read_csv(DATADIR / "split.csv")
train = train.merge(pre, on="StudyInstanceUID")

portion = pd.read_csv(DATADIR / "study_pos_portion.csv")
train = train.merge(portion, on="StudyInstanceUID")

z_pos_df = pd.read_csv(DATADIR / "sop_to_prefix.csv").rename(columns={'img_prefix': 'z_pos'})
train = train.merge(z_pos_df, on="SOPInstanceUID")

studies = train.StudyInstanceUID.unique()

In [24]:
oof = pd.concat([oof_f0, oof_f1, oof_f2, oof_f3, oof_f4]).rename(columns={'pe_present_on_image': 'pred'})

train = train.merge(oof[['pred', 'SOPInstanceUID']], on="SOPInstanceUID")  # add pred

In [25]:
from functools import partial
def grouping(df):
    grouped = pd.DataFrame(df.groupby('StudyInstanceUID')['pred'].mean())
    grouped = grouped.rename(columns={'pred': 'mean'})
    count = df.groupby('StudyInstanceUID')['pred'].count()
    grouped['count_total'] = count

    for i in range(1,10):
        count = df[df.pred>i/10].groupby('StudyInstanceUID')['pred'].count()
        grouped[f'count_over{i/10}'] = count
        grouped[f'count_over{i/10}_ratio'] = count / grouped['count_total']

    for q in [30, 50, 70, 80, 90, 95, 99]:
    # for q in [95]:
        grouped[f'percentile{q}'] = df.groupby('StudyInstanceUID')['pred'].apply(lambda arr: np.percentile(arr, q))

    ma = pd.DataFrame(df.groupby('StudyInstanceUID')['pred'].max())
    grouped['max'] = ma.pred

    grouped = grouped.reset_index().fillna(0)
    return grouped
train_grouped = grouping(train)

train_grouped['fold'] = train.groupby('StudyInstanceUID')['fold'].first().values
train_grouped['negative_exam_for_pe'] = train.groupby('StudyInstanceUID')['negative_exam_for_pe'].first().values
train_grouped['positive_exam_for_pe'] = (1 - train.groupby('StudyInstanceUID')['negative_exam_for_pe'].first().values) * (1 - train.groupby('StudyInstanceUID')['indeterminate'].first().values)

# test_grouped = grouping(test)

In [26]:
# features = list(set(list(train_grouped)) - set(['StudyInstanceUID', 'negative_exam_for_pe']))
# target = 'negative_exam_for_pe'

### features = list(set(list(train_grouped)) - set(['StudyInstanceUID', 'positive_exam_for_pe', 'negative_exam_for_pe', 'fold']))
features = list(set(list(train_grouped)) - set(['StudyInstanceUID', 'positive_exam_for_pe', 'negative_exam_for_pe', 'fold']) - set(['count_total']))
features = sorted(features)
target = 'positive_exam_for_pe'
# target = 'negative_exam_for_pe'

In [27]:
print(features)

['count_over0.1', 'count_over0.1_ratio', 'count_over0.2', 'count_over0.2_ratio', 'count_over0.3', 'count_over0.3_ratio', 'count_over0.4', 'count_over0.4_ratio', 'count_over0.5', 'count_over0.5_ratio', 'count_over0.6', 'count_over0.6_ratio', 'count_over0.7', 'count_over0.7_ratio', 'count_over0.8', 'count_over0.8_ratio', 'count_over0.9', 'count_over0.9_ratio', 'max', 'mean', 'percentile30', 'percentile50', 'percentile70', 'percentile80', 'percentile90', 'percentile95', 'percentile99']


In [28]:
train_grouped.head()

,StudyInstanceUID,mean,count_total,count_over0.1,count_over0.1_ratio,count_over0.2,count_over0.2_ratio,count_over0.3,count_over0.3_ratio,count_over0.4,...,percentile50,percentile70,percentile80,percentile90,percentile95,percentile99,max,fold,negative_exam_for_pe,positive_exam_for_pe
0,0003b3d648eb,0.271391,223,121.0,0.542601,94.0,0.421525,80.0,0.358744,61.0,...,0.125400,0.373026,0.589021,0.825415,0.907900,0.963111,0.971206,0,1,0
1,000f7f114264,0.091386,239,65.0,0.271967,45.0,0.188285,28.0,0.117155,16.0,...,0.006931,0.075649,0.177131,0.311252,0.457845,0.573354,0.889981,3,1,0
2,00102474a2db,0.055245,326,58.0,0.177914,21.0,0.064417,10.0,0.030675,4.0,...,0.015041,0.047743,0.089943,0.159699,0.251680,0.518155,0.652430,3,1,0
3,0038fd5f09f5,0.054540,230,34.0,0.147826,13.0,0.056522,6.0,0.026087,6.0,...,0.014543,0.056811,0.073162,0.130984,0.203891,0.503386,0.944670,4,1,0
4,0045f113e031,0.133363,257,111.0,0.431907,75.0,0.291829,45.0,0.175097,22.0,...,0.034198,0.186576,0.260993,0.380261,0.478244,0.765378,0.901021,4,1,0


In [29]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter('ignore')
import pickle
from sklearn.model_selection import KFold, StratifiedKFold

oof_preds_list = []
test_preds_list = []
models_list = []

for i in range(1):
    print(f'=================={i}================')
    if i % 4 == 0:
        params = {'boosting_type': 'gbdt',
            'objective': 'binary',
#             'metric': 'None',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.1,
            'feature_fraction': 0.9,
            'max_depth': 15,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'verbose': 100,
            'early_stopping_rounds': 100,
            'verbose': -1,
            } 
    elif i % 4 == 1:
        params = {
            'max_depth': 4,
            'max_leave': int(0.2 * 2 ** 4),
            'reg_lambda': 1,
            'reg_alpha': 1,
            'subsamples': 0.8,
            'colsample_bytree': 0.7,
            'objective': 'binary',
            'min_data_in_leaf': 0,
            'boosting': 'gbdt',
#             'metric': 'None',
            'learning_rate': 0.1,
                      }
    elif i % 4 == 2:
        params = {
            'num_leaves': 19, 
            'min_data_in_leaf': 160,
            'min_child_weight': 0.03,
            'bagging_fraction' : 0.7,
            'feature_fraction' : 0.8,
            'learning_rate' : 0.1,
            'max_depth': -1,
            'reg_alpha': 0.02,
            'reg_lambda': 0.12,
            'objective': 'binary',
            'verbose': 100,
            'boost_from_average': False,
#             'metric': 'None',
        }  
    else:
        params = {
            'objective': "binary",
#             'metric': 'None',
            'boost_from_average': "false",
            'tree_learner': "serial",
            'max_depth': -1,
            'learning_rate': 0.1,
            'num_leaves': 197,
            'feature_fraction': 0.3,
            'bagging_freq': 1,
            'bagging_fraction': 0.7,
            'min_data_in_leaf': 100,
            'bagging_seed': 11,
            'max_bin': 255,
            'verbosity': -1}    
        
    oof_preds = np.zeros(train_grouped.shape[0])

    ### test_preds = np.zeros(test_grouped.shape[0])
    val_results = {}
    models = []
    params['random_state'] = i
    iter = 100000
    kf = KFold(n_splits=5, shuffle=True, random_state=72)

    #for n_fold, (trn_idx, val_idx) in enumerate(kf.split(train_grouped, train_grouped[target])):
        # tr = train_grouped.iloc[trn_idx]
        # val = train_grouped.iloc[val_idx]
    for n_fold in range( 5 ):
        tr = train_grouped[train_grouped.fold != n_fold]
        val = train_grouped[train_grouped.fold == n_fold]
        trn_data = lgb.Dataset(tr[features], label=tr[target])
        val_data = lgb.Dataset(val[features], label=val[target])
        
        clf = lgb.train(params, trn_data, num_boost_round=iter, valid_sets=[trn_data, val_data], valid_names=['train', 'val'],
#                         feval=feval, verbose_eval=10, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
                        verbose_eval=200, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
        file = f'lgbs/posexam_lgb_seed{i}_fold{n_fold}.pkl'
        pickle.dump(clf, open(file, 'wb'))
        models.append(clf)
        
        oof_preds[train_grouped.fold == n_fold] = clf.predict(val[features])
        ### oof_preds[val_idx] = clf.predict(val[features])
        ### test_preds += clf.predict(test_grouped[features]) / 5
#     models_list.append(models)
    oof_preds_list.append(oof_preds)
    ###test_preds_list.append(test_preds)

print(f'--------------------------------------------------------------------roc: {roc_auc_score(train_grouped[target], np.mean(oof_preds_list, axis=0))}')
print(f'----------------------------------------------------------------logloss: {log_loss(train_grouped[target], np.mean(oof_preds_list, axis=0))}')
lgb_oof_exam = np.mean(oof_preds_list, axis=0)
### lgb_preds = np.mean(test_preds_list, axis=0)

==================0================
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[26]	train's binary_logloss: 0.272409	val's binary_logloss: 0.343537
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[43]	train's binary_logloss: 0.242331	val's binary_logloss: 0.30137
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[34]	train's binary_logloss: 0.253284	val's binary_logloss: 0.326764
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 roun

In [30]:
bce_func = torch.nn.BCELoss(reduction='mean')
lgb_losses = bce_func(torch.FloatTensor(oof_preds), torch.FloatTensor(train_grouped['positive_exam_for_pe']))

torch.mean(lgb_losses).item()

0.3298527002334595

In [31]:
bce_func = torch.nn.BCELoss(reduction='mean')
lgb_losses = bce_func(
    ( 1 - torch.FloatTensor(oof_preds) ) * (4911) / (4911 + 157), 
    torch.FloatTensor(train_grouped['negative_exam_for_pe']))

torch.mean(lgb_losses).item()

0.36254268884658813

In [32]:
# # current yama's pipeline for fold0-ep1
# def calib_p(arr, factor):  # set factor>1 to enhance positive prob
#     return arr * factor / (arr * factor + (1-arr))
# def post_yama(arr):
#     return calib_p( np.percentile(arr, 95), factor=1/8.5550)

# lgb_losses = bce_func(torch.FloatTensor(train[['StudyInstanceUID','pred']].groupby('StudyInstanceUID').apply(post_yama)), torch.FloatTensor(train_grouped['positive_exam_for_pe'])).item()
# lgb_losses